# CybORG quickstart
This notebook demonstrates basic operations: reloading the local `CybORG` package during development, creating an Enterprise scenario, instantiating the environment with a reproducible NumPy `Generator` seed, and performing a few simple calls (reset, step, get_true_state, close).

In [ ]:
# Input: set number of blue and red agents via terminal (fallback for non-interactive runs)
import os
try:
    blue_count = int(input("Enter number of blue agents (default 0): ") or 0)
    red_count = int(input("Enter number of red agents (default 0): ") or 0)
except Exception:
    # Non-interactive environment (nbconvert) - use env vars or defaults
    blue_count = int(os.environ.get("BLUE_COUNT", 0))
    red_count = int(os.environ.get("RED_COUNT", 0))

print(f"\nSelected: blue={blue_count}, red={red_count}")


Selected: blue=1, red=1


In [16]:
# Ensure local CybORG repo is importable and reload modules so edits are picked up in this kernel
import sys

repo_root = r'c:\Users\Student\cage-challenge-4-1'
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

# Remove any previously loaded CybORG modules so we import fresh copies after edits
mods_to_remove = [m for m in list(sys.modules) if m.startswith('CybORG')]
for mod in mods_to_remove:
    del sys.modules[mod]

# Import the package and helper generator
import CybORG
from CybORG.Simulator.Scenarios.EnterpriseScenarioGenerator import EnterpriseScenarioGenerator

print('Modules reloaded successfully')

Modules reloaded successfully


In [17]:
# Minimal example: create scenario, env (with reproducible seed), reset, step, and close
import numpy as np
from CybORG import CybORG
from CybORG.Simulator.Scenarios.EnterpriseScenarioGenerator import EnterpriseScenarioGenerator

# Create scenario generator and a NumPy Generator seed (recommended for reproducibility)
sg = EnterpriseScenarioGenerator()
rng = np.random.default_rng(12345)  # deterministic generator

# Wrap the generator's create_scenario so we can trim agents to user-specified counts
orig_create = sg.create_scenario

def create_scenario_with_counts(np_random):
    scenario = orig_create(np_random)

    def trim_team(team_name: str, desired_count: int):
        # collect agent names belonging to the team
        members = [name for name, info in list(scenario.agents.items()) if info.team == team_name or team_name.lower() in name.lower()]
        if len(members) <= desired_count:
            return
        # keep the first desired_count, remove the rest
        to_remove = members[desired_count:]
        for a in to_remove:
            scenario.agents.pop(a, None)
        # update team_agents mapping
        if team_name in scenario.team_agents:
            scenario.team_agents[team_name] = [a for a in scenario.team_agents[team_name] if a in scenario.agents]

    # trim each team according to user inputs (blue_count, red_count).
    # Also remove any Green agents so only Blue and Red remain.
    try:
        trim_team('Blue', int(blue_count))
        trim_team('Red', int(red_count))
        # remove all Green agents (we only want Red/Blue in this notebook)
        trim_team('Green', 0)
    except Exception:
        # if counts are invalid, ignore trimming
        pass

    return scenario

# Apply the wrapper
sg.create_scenario = create_scenario_with_counts

# Instantiate the environment (simulator backend)
env = CybORG(sg, 'sim', seed=rng)

# List agents (external-facing)
print('Agent ids:', env.get_agent_ids())

# Reset environment to get the true-state / initial global observation
obs = env.reset(agent=None)  # agent=None returns true-state / initial state
print('Initial true state keys:', list(obs.observation.keys())[:8])

# Single-agent interaction: pick the first agent and reset for that agent
agent = env.get_agent_ids()[0] if len(env.get_agent_ids())>0 else None
if agent:
    result = env.reset(agent=agent)
    # Print a compact summary of the action space (top-level keys) to avoid huge dumps
    if hasattr(result, 'action_space') and result.action_space is not None:
        try:
            keys = list(result.action_space.keys())
        except Exception:
            keys = []
        print('Initial action_space keys (sample):', keys[:20])
    else:
        print('No action_space returned for agent')
    
    # Take a default/no-op action if unsure (many tests use action=None)
    res = env.step(agent=agent, action=None)
    print('Step result: reward=', getattr(res, 'reward', None), ' done=', getattr(res, 'done', None))
else:
    print('No agents available to step')

# Example true-state query (returns a dict-like object)
true_state = env.get_true_state({'all': {'Interfaces': 'All'}})
print('True-state keys (sample):', list(true_state.keys())[:8])

Agent ids: ['blue_agent_0', 'red_agent_0']
Initial true state keys: ['success', 'restricted_zone_a_subnet_router', 'restricted_zone_a_subnet_user_host_0', 'restricted_zone_a_subnet_user_host_1', 'restricted_zone_a_subnet_user_host_2', 'restricted_zone_a_subnet_user_host_3', 'restricted_zone_a_subnet_user_host_4', 'restricted_zone_a_subnet_server_host_0']
Initial action_space keys (sample): ['action', 'allowed_subnets', 'subnet', 'ip_address', 'session', 'username', 'password', 'process', 'port', 'target_session', 'agent', 'hostname']
Step result: reward= 0  done= False
True-state keys (sample): ['success']
Initial action_space keys (sample): ['action', 'allowed_subnets', 'subnet', 'ip_address', 'session', 'username', 'password', 'process', 'port', 'target_session', 'agent', 'hostname']
Step result: reward= 0  done= False
True-state keys (sample): ['success']


In [18]:
# Compact team breakdown and close environment
# Count agents by team in the current scenario (Blue/Red)
agents = env.get_agent_ids()
counts = {'Blue': 0, 'Red': 0}
for a in agents:
    if a.startswith('blue_') or a.startswith('blue_agent') or 'blue' in a:
        counts['Blue'] += 1
    elif a.startswith('red_') or a.startswith('red_agent') or 'red' in a:
        counts['Red'] += 1
print('Agent team counts:', counts)

# Now close the environment
try:
    env.close()
    print('Environment closed')
except Exception as e:
    print('Error when closing environment:', e)

Agent team counts: {'Blue': 1, 'Red': 1}
Environment closed


In [ ]:
obs = env.reset(agent=env.get_agent_ids()[0])
print(obs.observation)
for hostname, host_info in obs.observation.items():
    if isinstance(host_info, dict):
        interfaces = host_info.get("Interface")
        sessions = host_info.get("Sessions")
        system = host_info.get("System info")
        print(f"{hostname}: interfaces={interfaces}, sessions={sessions}, system={system}")
    else:
        # Safe handling for non-subscriptable values (e.g. TernaryEnum)
        print(f"{hostname}: state={host_info}")

{'success': <TernaryEnum.UNKNOWN: 2>, 'restricted_zone_a_subnet_router': {'Interface': [{'interface_name': 'eth0', 'ip_address': IPv4Address('10.0.214.164'), 'Subnet': IPv4Network('10.0.214.0/24')}], 'Sessions': [{'session_id': 1, 'username': 'ubuntu', 'timeout': 0, 'PID': 1, 'Type': <SessionType.UNKNOWN: 1>, 'agent': 'blue_agent_0'}], 'Processes': [{'PID': 1, 'username': 'ubuntu'}], 'User Info': [{'username': 'root', 'Groups': [{'GID': 0}]}, {'username': 'user', 'Groups': [{'GID': 1}]}], 'System info': {'Hostname': 'restricted_zone_a_subnet_router', 'OSType': <OperatingSystemType.LINUX: 3>, 'OSDistribution': <OperatingSystemDistribution.UBUNTU: 8>, 'OSVersion': <OperatingSystemVersion.UNKNOWN: 1>, 'Architecture': <Architecture.x64: 2>, 'position': array([0., 0.])}}, 'restricted_zone_a_subnet_user_host_0': {'Interface': [{'interface_name': 'eth0', 'ip_address': IPv4Address('10.0.214.189'), 'Subnet': IPv4Network('10.0.214.0/24')}], 'Sessions': [{'session_id': 2, 'username': 'ubuntu', 't